### Assignment- III Solution 
#### Python script to provide execution analysis for the forward looking Market Making strategy

In [1]:
#Importing required libraries
import datetime
import timeit
import pandas as pd
import traceback
import matplotlib.pyplot as pp


## Index
  1. [Strategy Flow](#Strategy-Flow)
  2. [Forward Looking Market Making Simulation](#Market-Making-Simulation)
  3. [Output-Trades in Dataframe](#Display-Simulated-Trades)

### Strategy Flow

1. **Data Download** [Data Download](#Data-Download)
2. **Follow [Market Making Simulation Thesis](#Market-Making-Thesis)** to [Simulate Forward Looking Market Making](#Market-Making-Simulation)
3. **[Display Trades in Dataframe](#Display-Simulated-Trades)**   
3. **[Execution Analysis](#Execution-Analysis)**   

### Data Download

In [2]:
#Helper functions


def get_dates(start_date=None,end_date=None,n_days=None):
    '''
    Function to create n sequential dates
    
    Params:
    
    start_date: start date in the format '%Y-%m-%d' in UTC timezone
    end_date:  end date in the format '%Y-%m-%d' in UTC timezone
    n_days:    number of recent days
    
    Mandatory Params: 
    
    (start_date and end_date) or n_days
    
    Precedence of Params:
    n_days has higher precedence over (start_date and end_date)
    '''    

    if n_days is None:
        if start_date is None or end_date is None:
            return "Download Failed: Mandatory params missing"
        else:    
            start_date_dt=datetime.datetime.strptime(start_date,'%Y-%m-%d')
            end_date_dt=datetime.datetime.strptime(end_date,'%Y-%m-%d')
            days=0
            dates=[]


            while (start_date_dt+datetime.timedelta(days=days))<=end_date_dt:
                dates.append((start_date_dt+datetime.timedelta(days=days)).strftime('%Y-%m-%d'))
                days+=1
            return dates
    else:
        utcnow=datetime.datetime.utcnow()
        utclastday=(utcnow-datetime.timedelta(days=1))
        days=0
        dates=[]
        while days<=n_days:
            dates.append((utclastday+datetime.timedelta(days=-1*days)).strftime('%Y-%m-%d'))
            days+=1
        return dates[::-1]
            

def download_data(days):
    '''
    Function to download trade data from binance open source end point for given list of days
    
    Params:
    days: List of days in format '%Y-%m-%d' in UTC timezone
    '''
    if type(days)!=list:
        return "Download Failed"
    print("Data Downloading for : " ,days)
    frames=[]
    utclastday=''
    for utclastday in days:
        try:
            url='https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-{0}.zip'.format(utclastday)
            print(url)
            df=pd.read_csv(url,compression='zip',header=None)
            df['symbol']='BTCUSDT'
            df.columns=['id','price','qty','quote_qty','time','is_buyer_maker','symbol']
            df=df[['id','symbol','price','qty','quote_qty','time','is_buyer_maker']]
            df.set_index('id',inplace=True)
            frames.append(df)
        except Exception as ex:
            
            print(ex)
            print("Download Failed for ",utclastday)
    
    return "Couldn't download data for days" if len(frames)==0 else pd.concat(frames)      


In [3]:
#Downloader function
def data_downloader(start_date=None,end_date=None,n_days=None):
    '''
    Function to download trade data from binance open source end point
    
    Params:
    
    start_date: start date in the format '%Y-%m-%d' in UTC timezone from which the data hast to be downloaded
    end_date:  end date in the format '%Y-%m-%d' in UTC timezone to which the data hast to be downloaded
    n_days:    number of recent days for which the data has to be downloaded 
    
    Mandatory Params: 
    
    (start_date and end_date) or n_days
    
    Precedence of Params:
    n_days has higher precedence over (start_date and end_date)
    '''
    if n_days is None:
        if start_date is None or end_date is None:
            return "Download Failed: Mandatory params missing"
        else:
            return download_data(get_dates(start_date,end_date))
    else:
        return download_data(get_dates(n_days=n_days))


#### Downloading data for the desired dates


In [4]:
trade_data=data_downloader(start_date='2022-03-01',end_date='2022-03-03')
print(trade_data)


Data Downloading for :  ['2022-03-01', '2022-03-02', '2022-03-03']
https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-2022-03-01.zip
https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-2022-03-02.zip
https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-2022-03-03.zip
             symbol    price    qty  quote_qty           time  is_buyer_maker
id                                                                           
1987119092  BTCUSDT  43155.0  0.001      43.15  1646092800000            True
1987119093  BTCUSDT  43150.8  0.001      43.15  1646092800027            True
1987119094  BTCUSDT  43151.4  0.019     819.87  1646092800089            True
1987119095  BTCUSDT  43151.4  0.031    1337.69  1646092800089            True
1987119096  BTCUSDT  43151.4  0.001      43.15  1646092800089            True
...             ...      ...    ...        ...            ...             ...
1999773848  BTCUSDT  424

### Market Making Simulation

In [5]:
##Variable Parameters

trade_bps=2 #basis points for bid and ask orders
hedge_bps=0.5 #basis points for hedge orders (Ideally < trade basis points)

max_order_value=1000 #Order value in USD which is to be used for bid and ask quotes
hedge_threshold=5 #Hedge threshold in multiplier to max_order_value, which triggers passive hedging when a unidirectional-position value exceeds this value


In [6]:
def MM_Simulation(trade_data_input,trade_bps,hedge_bps,max_order_value,hedge_threshold,portfolio=10000):
    
    
    #Track start time of the function call
    st=timeit.default_timer()
    
    #copy the dataframe to temp dataframe
    trade_data = trade_data_input.copy(deep=True)
    
    #Convert input bps to a multiplier
    trade_bps=trade_bps*0.001
    hedge_bps=hedge_bps*0.001


    #Create bid and ask for next trade
    trade_data['my_bid']=(1-trade_bps)*trade_data['price']
    trade_data['my_ask']=(1+trade_bps)*trade_data['price']

    #Create imaginary hedge bid and ask values, to be used when hedging is required
    trade_data['my_hedge_bid']=(1-hedge_bps)*trade_data['price']
    trade_data['my_hedge_ask']=(1-hedge_bps)*trade_data['price']


    #Create Buy and sell quantities based on bid and ask prices
    trade_data['notional_capacity_buy']=(max_order_value)/trade_data['my_bid']
    trade_data['notional_capacity_sell']=(max_order_value)/trade_data['my_ask']


    #Shift bid and ask, which can be accesed by the next row for validation
    trade_data['prev_bid']=trade_data['my_bid'].shift(1)
    trade_data['prev_ask']=trade_data['my_ask'].shift(1)

    #Shift hedge bid and ask, which can be accesed by the next row for validation
    trade_data['prev_hedge_bid']=trade_data['my_hedge_bid'].shift(1)
    trade_data['prev_hedge_ask']=trade_data['my_hedge_ask'].shift(1)

    #Shift buy and sell quantities, which can be accesed by the next row for validation
    trade_data['notional_capacity_buy']=trade_data['notional_capacity_buy'].shift(1)
    trade_data['notional_capacity_sell']=trade_data['notional_capacity_sell'].shift(1)

    #Convert datetime to user readable format
    trade_data['date_time']=pd.to_datetime(trade_data.time, unit='ms',utc=True)

    #Drop unsued Columns
    trade_data.drop(['symbol','quote_qty','time','is_buyer_maker'],axis=1,inplace=True)
    
    trades=[] #trades list to hold trades as dictionaries
    open_position=0 #Open position quantity
    hedge_flag=0 #temporary flag which is used to track hedge trigger
    pos_avg_price=0 #Average entry price of open position, used to track PL

    for row in trade_data.itertuples(): #Iterate though dataframe as tuples to enhance execution speed

        #store values in meaningful variable names
        
        price = row[1]
        qty=row[2]
        prev_bid_qty=row[7]
        prev_ask_qty=row[8]
        prev_bid=row[9]
        prev_ask=row[10]
        prev_hedge_bid=row[11]
        prev_hedge_ask=row[12]
        date=row[13]
        prev_hedge_flag=hedge_flag


        #Buy bid validation
        #If hedge flag is not triggered in previous iteration, validate previous bid to check long position entry
        if prev_hedge_flag!=1:
            if price<=prev_bid:
                #validate last traded qty with bid quantity to choose realistic quantity 
                trade_qty=qty if qty<prev_bid_qty else prev_bid_qty 
                #create dictionary of collectables from the trade
                trade={'date_time':date,"trade_type":"Buy","qty":trade_qty ,"trade_price":prev_bid}
                
                #Calculate PL if the trade is in opposite direction to current position else adjust entry price of open position price 
                pl=0.0
                if open_position>=0:  
                    pos_avg_price=((trade_qty*prev_bid)+(pos_avg_price*open_position))/(trade_qty+open_position)
                else:
                    pl=((pos_avg_price-prev_bid)*trade_qty)
                    if open_position+trade_qty>0:
                        pos_avg_price=prev_bid
                trade['PL']=pl
                #adjust open position
                open_position+=trade_qty
                
                #Adjust portfolio value and create estimated portfolio value
                portfolio-=(trade_qty)*trade['trade_price']
                un_settled_value=(open_position*price)
                trade['open_position']=open_position
                trade['portfolio_value']=portfolio+un_settled_value

                trades.append(trade)
        #If hedge flag is triggered for buy side, trigger passive hedging for sell
        else:
            if price>=prev_hedge_ask:
                trade_qty=qty if qty<open_position or open_position==0.0 else open_position
                trade={'date_time':date,"trade_type":"Hedge Sell","qty":trade_qty ,"trade_price":prev_hedge_ask}

                pl=0.0
                if open_position<=0:  
                    pos_avg_price=((trade_qty*prev_hedge_ask)+(pos_avg_price*abs(open_position)))/(trade_qty+abs(open_position))
                else:
                    pl=((prev_hedge_ask-pos_avg_price)*trade_qty)
                    if open_position+trade_qty<0:
                        pos_avg_price=prev_hedge_ask
                trade['PL']=pl

                open_position-=trade_qty
                portfolio+=(trade_qty)*trade['trade_price']
                un_settled_value=(open_position*price)
                trade['open_position']=open_position
                trade['portfolio_value']=portfolio+un_settled_value
                trades.append(trade)    
        
        #Vice versa of Buy bid validation to validate sell orders
        if prev_hedge_flag!=-1:       
            if price>=prev_ask:
                trade_qty=qty if qty<prev_ask_qty else prev_ask_qty
                trade={'date_time':date,"trade_type":"Sell","qty":trade_qty ,"trade_price":prev_ask}

                pl=0.0
                if open_position<=0:  
                    pos_avg_price=((trade_qty*prev_ask)+(pos_avg_price*abs(open_position)))/(trade_qty+abs(open_position))
                else:
                    pl=((prev_ask-pos_avg_price)*trade_qty)
                    if open_position+trade_qty<0:
                        pos_avg_price=prev_ask
                trade['PL']=pl

                open_position-=trade_qty
                portfolio+=(trade_qty)*trade['trade_price']
                un_settled_value=(open_position*price) 
                trade['open_position']=open_position
                trade['portfolio_value']=portfolio+un_settled_value
                trades.append(trade)            
        else:
            if price<=prev_hedge_bid:
                trade_qty=qty if qty<abs(open_position) or  open_position==0.0  else abs(open_position)
                trade={'date_time':date,"trade_type":"Hedge Buy","qty":trade_qty ,"trade_price":prev_hedge_bid}

                pl=0.0
                if open_position>=0:  
                    pos_avg_price=((trade_qty*prev_hedge_bid)+(pos_avg_price*open_position))/(trade_qty+open_position)
                else:
                    pl=((pos_avg_price-prev_hedge_bid)*trade_qty)
                    if open_position+trade_qty>0:
                        pos_avg_price=prev_hedge_bid
                trade['PL']=pl

                open_position+=trade_qty
                portfolio-=(trade_qty)*trade['trade_price']
                un_settled_value=(open_position*price) 
                trade['open_position']=open_position
                trade['portfolio_value']=portfolio+un_settled_value
                trades.append(trade)            

        if open_position>0 and open_position < prev_bid_qty*hedge_threshold:
            hedge_flag=0
        elif open_position<0 and abs(open_position) < prev_ask_qty*hedge_threshold:
            hedge_flag=0
        elif open_position>0 and open_position > prev_bid_qty*hedge_threshold:
            hedge_flag=1
        elif open_position<0 and abs(open_position) > prev_ask_qty*hedge_threshold:
            hedge_flag=-1 
        
    print("Execution time in Seconds: ",timeit.default_timer()-st)
    return pd.DataFrame(trades)

In [7]:
simulated_trades = MM_Simulation(trade_data,trade_bps,hedge_bps,max_order_value,hedge_threshold,portfolio=10000)

Execution time in Seconds:  107.6428926


### Display Simulated Trades

In [8]:
simulated_trades

,date_time,trade_type,qty,trade_price,PL,open_position,portfolio_value
0,2022-03-01 08:22:04.524000+00:00,Buy,0.022919,43632.5600,0.000000,0.022919,9999.308773
1,2022-03-01 11:26:08.473000+00:00,Buy,0.015000,43812.2000,0.000000,0.037919,10003.692420
2,2022-03-01 11:26:08.475000+00:00,Sell,0.020000,43811.7486,2.162519,0.017919,10005.681340
3,2022-03-01 11:26:08.475000+00:00,Buy,0.022825,43812.2000,0.000000,0.040743,10000.526740
4,2022-03-01 11:26:08.475000+00:00,Sell,0.022825,43810.9470,1.061347,0.017918,10005.437666
...,...,...,...,...,...,...,...
613,2022-03-03 15:36:59.619000+00:00,Sell,0.023444,42654.0378,0.000000,-0.078993,10106.309974
614,2022-03-03 15:36:59.621000+00:00,Buy,0.003000,42624.3804,0.067237,-0.075993,10117.273646
615,2022-03-03 15:36:59.623000+00:00,Sell,0.010000,42654.0378,0.000000,-0.085993,10108.528205
616,2022-03-03 15:36:59.626000+00:00,Buy,0.016000,42639.1508,0.135753,-0.069993,10117.001012


###  Execution Analysis

In [9]:
def max_draw_down(input_data,date):
    '''
    Function to calculate max draw down of given data
    '''
    data=input_data.copy(deep=True)
    data=data.loc[data.date==date]
    roll_max = data['portfolio_value'].cummax()
    drawdown = data['portfolio_value']/roll_max - 1.0
    max_drawdown = drawdown.cummin() 
    return (max_drawdown.min())

In [10]:

def get_stats(trades):
    '''
    Function to get trade stats
    
    '''
    
    #copy data to temp data frame
    simulated_trades=trades.copy(deep=True)
    
    #fetch date of each row
    simulated_trades['date']=simulated_trades['date_time'].dt.date

    #Flag winning trades
    simulated_trades['iswin']=0
    simulated_trades.loc[(simulated_trades.PL>0),'iswin']=1
    
    #Find winning and loosing PL
    simulated_trades['positivePL']=0.0
    simulated_trades.loc[simulated_trades.iswin==1,'positivePL']=simulated_trades.PL
    simulated_trades['negativePL']=0.0
    simulated_trades.loc[simulated_trades.iswin==0,'negativePL']=simulated_trades.PL
    
    
    #Aggregate by date
    aggregated_simulated_trades  = simulated_trades.groupby('date').agg({'PL': 'sum', 'trade_type': 'count','positivePL':'sum','negativePL':'sum','qty':'sum','iswin':['sum','count']})
    
    aggregated_simulated_trades.columns=['GrossPL$','Number_Of_trades','PositivePL','NegativePL','Volume','NumProfitTrades','TotalNumTrades']
    
    #Calculate Gross Profit in BPS
    aggregated_simulated_trades['GrossBPS']=(aggregated_simulated_trades['GrossPL$']/aggregated_simulated_trades['Volume'])*10000
    
    #Calculate Max drawdown
    aggregated_simulated_trades['MaxDD']=0.0
    
    #Calculate Average win and loose PL
    aggregated_simulated_trades['Average_win_PL']=aggregated_simulated_trades['PositivePL']/aggregated_simulated_trades['NumProfitTrades']
    
    aggregated_simulated_trades['Average_loss_PL']=aggregated_simulated_trades['NegativePL']/(aggregated_simulated_trades['TotalNumTrades']-aggregated_simulated_trades['NumProfitTrades'])
                                                                                              
    
    #Calculate max drawdown
    for i in range(len(aggregated_simulated_trades)):
        aggregated_simulated_trades['MaxDD'].values[i]=max_draw_down(simulated_trades,aggregated_simulated_trades.index.values[i])

    #select desired rows and return dataframe
    return aggregated_simulated_trades[['GrossBPS','GrossPL$','MaxDD','Average_win_PL','Average_loss_PL','TotalNumTrades']]




In [11]:
result=get_stats(simulated_trades)

In [12]:
result

,GrossBPS,GrossPL$,MaxDD,Average_win_PL,Average_loss_PL,TotalNumTrades
date,,,,,,
2022-03-01,220426.128493,113.751334,-0.004017,0.898480,-0.156058,332
2022-03-02,593084.721933,147.361026,-0.006087,3.044386,-0.832227,149
2022-03-03,-536314.744109,-141.093839,-0.012434,0.659049,-1.911879,137


In [13]:
result.to_csv('summary.csv')